# Running BLAST on the Cloud

In this notebook we run a BLASTN search using an antibiotic resistance gene (MH168512) as the query and a set of E. coli plasmids as the database.  We'll then use Python and the Pandas module to explore the resulting data.

Below, we start out by retrieving the plasmid sequences and building a BLAST database from those.

To get started, click on the cell and then use shift-enter to run it or use the Run button above.

In [1]:
!curl -O https://ftp.ncbi.nlm.nih.gov/blast/demo/Plasmids_562.fsa

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  125M  100  125M    0     0  7573k      0  0:00:16  0:00:16 --:--:-- 11.5M


In [ ]:
!makeblastdb -in Plasmids_562.fsa -dbtype nucl -parse_seqids -taxid 562 -out Plasmids_562

In the makeblastdb command (above), we produced a BLAST database from the plasmid FASTA file, named it Plasmids_562, and set the taxid (taxonomy) for every entry to 562 (E. coli).

Next, we download the query file for the antibiotic resistance gene.

In [ ]:
!curl -O https://ftp.ncbi.nlm.nih.gov/blast/demo/MH168512.fsa

We'll run a BLASTN search and format the results as a table.  The query is our antibiotic resistance gene (MH168512) and the database is the set of plasmid sequences. We're including a lot of fields in the table so we can later use a python script to read it into a pandas dataframe.

In [ ]:
!blastn -db Plasmids_562 -query MH168512.fsa -outfmt "6 qseqid sseqid stitle pident qcovs length mismatch gapopen qstart qend sstart send qframe sframe frames evalue bitscore qseq sseq" -out MH168512.tab -max_target_seqs 5000

Check the output of the tabular results with the UNIX "head" command just to make sure something is there:

In [ ]:
!head MH168512.tab

Now we download a Python script that we use to load our tabular output into a Pandas dataframe.

In [ ]:
!curl -O https://raw.githubusercontent.com/fomightez/sequencework/master/blast-utilities/blast_to_df.py

Load the tabular results into a dataframe:

In [ ]:
%run blast_to_df.py MH168512.tab

We need one more step to get it into a dataframe we can query. We'll do that and then look at the top of first few lines of the table.

In [ ]:
import pandas as pd
df = pd.read_pickle("BLAST_pickled_df.pkl")
df.head()

We can use the dataframe to extract other information.  In the next two cells, we'll check how many rows are in the table, and how many unique database matches we found.

In [ ]:
df['sseqid'].count()

In [ ]:
df[df.duplicated('sseqid', 'first') != True]['sseqid'].count()

There were 81 rows in the table, but only 77 different database sequences (i.e., plasmids) were found. This indicates that some plasmids contained multiple copies of the AMR gene.  To confirm this, we'll need to go back and take a look at the blast results.  The next command will identify those plasmids with multiple BLAST matches and print them out.

In [ ]:
df[df.duplicated('sseqid', False)]

It looks like we have three plasmids with multiple BLAST matches.  Those three plasmids are in rows 20-22 (NZ_CP010373.2), 61-62 (NZ_CP025626.1), and 76-77 (NZ_CM007910.1). 

Looking at these results, we see that these are all strong matches.  All the matches are in excess of 99% identical (pident column) and they all cover most of the gene (length column).  The sstart and send columns identify the start and end of the alignment on the plasmid sequence.

This use of pandas was inspired by the workflow at https://github.com/fomightez/blast-binder  